In [1]:
import sqlitedatastore as datastore

datastore.connect()
# datastore.create_table()
datastore.close()

In [2]:
import glob
import json
import urllib.parse

import scrape
import sqlitedatastore as datastore

datastore.connect()
values = []
for filename in glob.glob('./data/wikipedia/*.html'):
    with open(filename) as fin:
        html = fin.read()
        text, title = scrape.scrape(html)
        print('scraped:', title)
        url = 'https://ja.wikipedia.org/wiki/{0}'.format(
            urllib.parse.quote(title))
        values.append((text, json.dumps({'url': url, 'title': title})))
datastore.load(values)

print(list(datastore.get_all_ids(limit=-1)))
datastore.close()

scraped: フィジー
scraped: ブラジル
scraped: 日本
scraped: ツバル
scraped: カメルーン
scraped: アメリカ合衆国
scraped: モーリシャス
scraped: ジンバブエ
scraped: セントクリストファー・ネイビス
scraped: イギリス
scraped: インド
scraped: フランス
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [4]:
import json

datastore.connect()
for doc_id in datastore.get_all_ids(limit=-1):
    row = datastore.get(doc_id, ['id', 'content', 'meta_info'])
    print(row['id'], json.loads(row['meta_info']), row['content'][:100])
datastore.close()


1 {'url': 'https://ja.wikipedia.org/wiki/%E3%83%95%E3%82%A3%E3%82%B8%E3%83%BC', 'title': 'フィジー'} フィジー<__EOS__>
フィジー共和国（フィジーきょうわこく）、通称フィジーは、オセアニアの国家で、イギリス連邦加盟国である。首都はビティレブ島のスバ。南太平洋のフィジー諸島と北に500km程離れ
2 {'url': 'https://ja.wikipedia.org/wiki/%E3%83%96%E3%83%A9%E3%82%B8%E3%83%AB', 'title': 'ブラジル'} ブラジル<__EOS__>
ブラジル手話語 <__EOS__>
ブラジル連邦共和国（ブラジルれんぽうきょうわこく、ポルトガル語: República Federativa do Brasil）、通称ブ
3 {'url': 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC', 'title': '日本'} 日本<__EOS__>
日本国（にほんこく、にっぽんこく、英: Japan）、または日本（にほん、にっぽん）は、東アジアに位置し、日本列島[注 2]および南西諸島・伊豆諸島・小笠原諸島などからなる民主
4 {'url': 'https://ja.wikipedia.org/wiki/%E3%83%84%E3%83%90%E3%83%AB', 'title': 'ツバル'} ツバル<__EOS__>
座標: 南緯8度31分0秒 東経179度12分0秒﻿ / ﻿南緯8.51667度 東経179.20000度﻿ / -8.51667; 179.20000 <__EOS__>

5 {'url': 'https://ja.wikipedia.org/wiki/%E3%82%AB%E3%83%A1%E3%83%AB%E3%83%BC%E3%83%B3', 'title': 'カメルーン'} カメルーン<__EOS__>
カメルーン共和国（カメルーンきょうわこく）、通称カメルーンは、中部アフリカに位置する共和制国家。西にナイジェリア、北東にチャド、東に中央アフリカ共和国、南東にコンゴ共和国
6 {'url': 'https://ja.wikipedi

In [5]:
import sqlitedatastore as datastore
import solrindexer     as indexer


datastore.connect()
data = []
for doc_id in datastore.get_all_ids(limit=-1):
    row = datastore.get(doc_id, ['id', 'content', 'meta_info'])
    # Solr へ登録するデータ構造へ変換
    meta_info = json.loads(row['meta_info'])
    data.append({
        'id':               str(row['id']),
        'doc_id_i':         row['id'],
        'content_txt_ja':   row['content'],
        'title_txt_ja':     meta_info['title'],
        'url_s':            meta_info['url'],
    })
# Solr への登録を実行
indexer.load('doc', data)
datastore.close()

{
  "responseHeader":{
    "status":0,
    "QTime":1133}}

{
  "responseHeader":{
    "status":0,
    "QTime":246}}

